# Ciencia de Datos - TP5

## Integrantes

- Ambroa, Nicolás - 229/13 - ambroanicolas@hotmail.com
- Gaustein, Diego - 586/09 - diego@gaustein.com.ar

In [109]:
from collections import Counter
from functools import wraps
from itertools import chain
from string import punctuation
import os
import pickle
from pprint import pprint
from lxml import html
from nltk import sent_tokenize, word_tokenize, pos_tag, WordNetLemmatizer
from nltk.corpus import wordnet, stopwords

lemmatizer = WordNetLemmatizer()
stopwords = frozenset(stopwords.words('english'))

def load_or_call(func):
    """ Decorador auxiliar para cachear resultados en un .pickle. """
    @wraps(func)
    def wrapper(*args, **kwargs):
        filename = '{}-{}-{}.pickle'.format(func.__name__, str(args), str(kwargs))
        if os.path.exists(filename):
            with open(filename, 'rb') as f:
                return pickle.load(f)
        else:
            res = func(*args, **kwargs)
            with open(filename, 'wb') as f:
                pickle.dump(res, f)
            return res

    return wrapper

### 1.1. Levantar el corpus AP, separando cada noticia como un elemento distinto en un diccionario `(<DOCNO>: <TEXT>)`.

In [3]:
parsed = html.parse('ap/ap.txt')
documents = {}
for document in parsed.iter('doc'):
    docno, article = document.getchildren()
    documents[docno.text.strip()] = article.text.strip()

print('Cargados', len(documents), 'articulos')

Cargados 2250 articulos


### 1.2. Calcular el tamaño del vocabulario.

In [111]:
def get_wordnet_pos(treebank_tag):
    if treebank_tag.startswith('J'):
        return wordnet.ADJ
    elif treebank_tag.startswith('V'):
        return wordnet.VERB
    elif treebank_tag.startswith('N'):
        return wordnet.NOUN
    elif treebank_tag.startswith('R'):
        return wordnet.ADV
    elif treebank_tag.startswith('S'):
        return wordnet.ADJ_SAT
    else:
        return wordnet.NOUN  # El default es NOUN

def get_words_for_document(document):
    for sentence in sent_tokenize(document):
        tagged_sentence = pos_tag(sentence.split())
        for word, pos in tagged_sentence:
            # Lematizar
            word = lemmatizer.lemmatize(word, pos=get_wordnet_pos(pos))
            # Strip punctuation
            word = word.strip(punctuation)
            # Lowercase
            word = word.lower()
            
            # Skip stopwords and punctuation
            if len(word) > 1 and word not in stopwords:
                yield word

@load_or_call
def get_word_count():
    c = Counter()
    for document in documents.values():
        c.update(get_words_for_document(document))
    return c
        
c = get_word_count()
cantidad_de_palabras = sum(c.values())
print(cantidad_de_palabras, 'palabras,', len(c), 'palabras distintas.')

548169 palabras, 41090 palabras distintas.


### 1.3. Para las 500 palabras con más apariciones, calcular el par más asociado según la medida presentada.

In [ ]:
"""
Para cada palabra, aproximamos p(palabra) con su frecuencia relativa según el counter.
Tomamos las 500 con más apariciones y recorremos el texto. Cada vez que encontremos una de estas miramos en una
ventana (n=8) cuando aparece alguna de las otras, y le sumamos uno.
Esta es nuestra estimación de p(palabra1, palabra2). Finalmente calculamos la información mutua e imprimimos los
pares más asociados.
"""
